In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
training_files = 3
data_files = glob.glob('play_data/*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['trick_num'] * 4 + train_data['play_seq']

In [3]:
train_data.head()

,run_id,key,trick_num,play_seq,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,...,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg,play_pos
0,953904,10♣ 9♥ A♣ J♥ K♣,5,1,0,0,0,1,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
1,953904,10♣ 9♥ J♥ K♣ A♣,5,1,0,0,0,0,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
2,953904,9♥ 10♣ J♥ A♣ K♣,5,0,0,0,0,1,5,3,...,0,0,0,2,2,2.0,-2,-2,-2.0,20
3,953904,10♣ J♥ 9♥ A♣ K♣,5,1,0,0,0,0,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
4,953904,10♣ J♥ K♣ 9♥ A♣,5,1,0,0,0,1,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21


In [4]:
label = 'points_max'
train_data[label].describe()

count    587716.000000
mean         -0.083217
std           1.644207
min          -4.000000
25%          -1.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_max, dtype: float64

In [5]:
exclude = ['run_id', 'key', 'trick_num', 'play_seq']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
outputs.remove(label)

type_map = {k: 'int' for k in list(train_data)}
type_map['play_pos'] = 'category'
type_map[label] = 'float'
metadata = FeatureMetadata(type_map_raw=type_map)

predictor = TabularPredictor(problem_type='regression', label=label)
predictor.fit(train_data.drop(columns=(exclude + outputs)), feature_metadata=metadata, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240904_215517"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       7.48 GB / 15.34 GB (48.7%)
Disk Space Avail:   123.42 GB / 452.95 GB (27.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoG

(_ray_fit pid=49560) [1000]	valid_set's rmse: 0.686219
(_ray_fit pid=49843) [1000]	valid_set's rmse: 0.68238
(_ray_fit pid=49842) [1000]	valid_set's rmse: 0.677559


(_dystack pid=48354) 	-0.6818	 = Validation score   (-root_mean_squared_error)
(_dystack pid=48354) 	131.66s	 = Training   runtime
(_dystack pid=48354) 	61.15s	 = Validation runtime
(_dystack pid=48354) Fitting model: LightGBM_BAG_L1 ... Training model for up to 6.56s of the 304.55s of remaining time.
(_dystack pid=48354) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 10.44% memory usage per fold, 41.78%/80.00% total).
(_dystack pid=48354) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=2, gpus=0, memory=10.44%)
(_dystack pid=48354) 	-1.2662	 = Validation score   (-root_mean_squared_error)
(_dystack pid=48354) 	10.01s	 = Training   runtime
(_dystack pid=48354) 	0.55s	 = Validation runtime
(_dystack pid=48354) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 291.41s of remaining time.
(_dystack pid=48354) 	Ensemble Weights: {'KNeighborsUnif_BAG_L1': 

(_ray_fit pid=50606) [1000]	valid_set's rmse: 0.464582 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=50607) [2000]	valid_set's rmse: 0.450142 [repeated 4x across cluster]
(_ray_fit pid=50983) [1000]	valid_set's rmse: 0.456329 [repeated 4x across cluster]
(_ray_fit pid=50985) [2000]	valid_set's rmse: 0.444955 [repeated 4x across cluster]


(_dystack pid=48354) 	-0.4453	 = Validation score   (-root_mean_squared_error)
(_dystack pid=48354) 	267.69s	 = Training   runtime
(_dystack pid=48354) 	236.53s	 = Validation runtime
(_dystack pid=48354) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -9.94s of remaining time.
(_dystack pid=48354) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 1.0}
(_dystack pid=48354) 	-0.4453	 = Validation score   (-root_mean_squared_error)
(_dystack pid=48354) 	0.44s	 = Training   runtime
(_dystack pid=48354) 	0.01s	 = Validation runtime
(_dystack pid=48354) AutoGluon training complete, total runtime = 906.12s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 184.7 rows/s (65302 batch size)
(_dystack pid=48354) TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240904_215517/ds_sub_fit/sub_fit_ho")
(_dystack pid=48354) Deleting DyStack predictor artifacts (clean_up_fits=True) ...
Leaderboard on holdout data (DySta

In [6]:
test_data = TabularDataset(data_files[training_files])[:50000]
test_data['play_pos'] = test_data['trick_num'] * 4 + test_data['play_seq']

y_pred = predictor.predict(test_data.drop(columns=([label] + exclude + outputs)))
y_pred.head()

Loaded data from: play_data/play_model_1-20240903_111103.tsv | Columns = 56 / 56 | Rows = 192843 -> 192843


0    0.990437
1   -1.764562
2   -1.144348
3    1.037158
4    1.005926
Name: points_max, dtype: float32

In [7]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -0.9840073533646305,
 'mean_squared_error': -0.9682704714756648,
 'mean_absolute_error': -0.5886288498151488,
 'r2': 0.551313831582523,
 'pearsonr': 0.7492442527078566,
 'median_absolute_error': -0.16913363337516785}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,-0.800801,-0.580895,root_mean_squared_error,60.833311,683.509781,480.453661,60.833311,683.509781,480.453661,1,True,3
1,WeightedEnsemble_L2,-0.974424,-0.500608,root_mean_squared_error,113.579120,1299.344462,481.500422,0.001868,0.005975,0.622770,2,True,4
2,LightGBMXT_BAG_L2,-0.984007,-0.345887,root_mean_squared_error,268.814327,2854.009228,1226.717956,106.913989,938.428380,745.446604,2,True,5
3,WeightedEnsemble_L3,-0.984007,-0.345887,root_mean_squared_error,268.816380,2854.014971,1227.190558,0.002053,0.005743,0.472602,3,True,6
4,KNeighborsUnif_BAG_L1,-1.428077,-0.658769,root_mean_squared_error,48.323086,616.242360,0.393699,48.323086,616.242360,0.393699,1,True,1
5,KNeighborsDist_BAG_L1,-1.430028,-0.598749,root_mean_squared_error,52.743941,615.828706,0.423991,52.743941,615.828706,0.423991,1,True,2
